# 7th Day of Python 4 DIP

> Contents:
> 1. Scikit-Image tools 
> 2. Histogram based image enchancements

Execute the cells to gain outputs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import skimage as SK
%matplotlib inline

In [ ]:
def ShowTime(
        Image,
        figsize=(12,7),
        bins=32,
        Im_title="Image",
        With_Hist=False,
        His_title="Histogram",
        Save=False,
        Save_Name="Day06",
        format='png',
        cmap='gray',
        colorbar=True
        ):
    """
    Awesome Image shower with histogram subplot
    With capability of saving Output as any format like {'png','pdf','eps',...}
    """
    if With_Hist:
            fig,ax = plt.subplots(figsize=figsize,nrows=1,ncols=2)
            fig.subplots_adjust(wspace=0.2)
            a = ax[0].imshow(Image,cmap=cmap)
            Loc = make_axes_locatable(ax[0])
            Cax = Loc.append_axes("bottom",size="5%",pad="2%")
            ax[0].set_title(Im_title)
            ax[0].axis('off')
            if colorbar:
                 fig.colorbar(a,cax=Cax,shrink=0.9,orientation="horizontal")


            #Hist,Bins = np.histogram(Image.ravel(),bins=range(bins))
            Hist,Bins = SK.exposure.histogram(Image,nbins=bins)
            CDF,_ = SK.exposure.cumulative_distribution(Image,nbins=bins)

            ax[1].bar(Bins,Hist,ec="black",fc="blue",width=2.0)
            Ex = ax[1].twinx()
            Ex.set_ylim(0,1.2)
            Ex.plot(Bins,CDF,'r',lw=2.0)
            ax[1].set_title(His_title)
            plt.show()
            if Save:
                fig.savefig(Save_Name+'.'+format,format=format)
    else:
         a = plt.imshow(Image,cmap=cmap)
         plt.axis('off')
         plt.title(Im_title)
         if colorbar:
              plt.colorbar(a,shrink=0.9,orientation="horizontal")
         plt.show()
         if Save:
            plt.savefig(Save_Name+'.'+format,format=format)




def ShowTimeMul(
    Images,
    figsize=(18, 7),
    bins=32,
    Im_title=None,
    With_Hist=False,
    His_Title=None,
    colorbar=True,
    cmap='gray'
):
    """
    Multi Image show time plotter.
    """
    n = len(Images)
    if With_Hist:
        fig, ax = plt.subplots(nrows=n, ncols=2, figsize=figsize)
        fig.subplots_adjust(wspace=0.2)

        for i in range(n):
            a = ax[i, 0].imshow(Images[i],cmap=cmap)
            ax[i, 0].axis("off")
            ax[i, 0].set_title(Im_title[i])
            Loc = make_axes_locatable(ax[i, 0])
            Cax = Loc.append_axes("bottom", size="5%", pad="2%")

            if colorbar:
                fig.colorbar(a, cax=Cax, shrink=0.9, orientation="horizontal")

            Hist, Bins = SK.exposure.histogram(Images[i], nbins=bins)
            CDF, _ = SK.exposure.cumulative_distribution(Images[i], nbins=bins)
            ax[i, 1].bar(Bins, Hist, ec="black", fc="blue")
            Ex = ax[i, 1].twinx()
            Ex.plot(Bins, CDF, "r", lw=2.5)
            ax[i, 1].set_title(His_Title[i])
        fig.tight_layout()
        plt.show()

    else:
        fig, ax = plt.subplots(nrows=1, ncols=n, figsize=figsize)

        for i in range(n):
            a = ax[i].imshow(Images[i],cmap=cmap)
            ax[i].axis("off")
            ax[i].set_title(Im_title[i])
            if colorbar:
                fig.colorbar(a, shrink=0.8, orientation="horizontal")
        fig.tight_layout()
        plt.show()

    

### Problem solved from saving issue.

In [ ]:
A = np.zeros((8,8))
A[0::2,1::2] = 1.0
A[1::2,0::2] = 1.0
ShowTime(A,With_Hist=False,colorbar=False,Im_title="Chessboard",Save=True,Save_Name="./Figures/Outputs/Chessboard",format="pdf")

### Using SK Data library

In [ ]:
Camera = SK.data.camera()
ShowTime(Camera,With_Hist=True,bins=256,colorbar=True)

### Contrast stretching

In [ ]:
#a,b = np.percentile(Camera,q=(10,90))
a,b = (20,200)
Rescaled_Camera = SK.exposure.rescale_intensity(Camera,in_range=(a,b))
ShowTime(Rescaled_Camera,bins=256,Im_title="Scaled version", With_Hist=True)

### Histogram equalization

In [ ]:
Equalized_Camera = SK.exposure.equalize_hist(Camera)*255
ShowTime(Equalized_Camera,bins=256,Im_title="Equalized version", With_Hist=True)

### Adaptive Equalization

In [ ]:
AdEqualized_Camera = SK.exposure.equalize_adapthist(Camera,clip_limit=0.5)*255
ShowTime(AdEqualized_Camera,bins=256,Im_title="Adaptive Equalized version", With_Hist=True)

In [ ]:
Gamma_Camera = SK.exposure.adjust_gamma(Camera,1.5)
ShowTime(Gamma_Camera,bins=256,Im_title="Gamma corrected", With_Hist=True)
print(f"Are you saw darker image?  {Gamma_Camera.mean() < Camera.mean()}")

In [ ]:
Log_Camera = SK.exposure.adjust_log(Camera,2)
ShowTime(Log_Camera,bins=256,Im_title="Log corrected", With_Hist=True)
print(f"Are you saw darker image?   {Log_Camera.mean() < Camera.mean()}")

### All Equalizations in one figure

In [ ]:
Images = (Camera,Rescaled_Camera,Equalized_Camera,AdEqualized_Camera,Log_Camera,Gamma_Camera)
ShowTimeMul(Images,bins=256,figsize=(13,18),Im_title=["Original","Rescaled","Equalized","Adaptive","Log","Gamma"],With_Hist=True,His_Title=["Histogram"]*len(Images),colorbar=True)

### Histogram matching

In [ ]:
Source = SK.io.imread("./Figures/cameraman.jpg",as_gray=True)*255
Refrence = SK.data.coffee()
Matched_Source = SK.exposure.match_histograms(Source,Refrence[:,:,2])
ShowTimeMul(Images=(Source,Refrence[:,:,1],Matched_Source),figsize=(12,16),bins=256,With_Hist=True,Im_title=["Source","Refrence","Matched"],His_Title=["Histogram"]*3)

### Now let Talk about padding 

In [ ]:
def ZeroPadding(Image,height,width):
    """
    The Zero padding module (First try)
    """
    M,N = Image.shape
    Padded = np.zeros((M+2*height,N+2*width))
    Padded[height:-height,width:-width] = Image
    return Padded

In [ ]:
ShowTimeMul((A,ZeroPadding(A,2,2)),With_Hist=False,colorbar=False,Im_title=["Original","Zero padded"])